In [7]:
#!pip3 install bs4
#!pip3 install requests
#!pip3 install newspaper3k
#!pip3 install -U selenium
#!pip3 install webdriver-manager
#!pip3 install pymongo[srv]
#!pip3 install pymongo[tls]
#!pip3 install lxml
import pymongo
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from pymongo import MongoClient

localConn = MongoClient('mongodb://127.0.0.1:27017')

db = localConn.concursos
tb = db.megasena


def getConcurso(numero_concurso):
    url = 'https://www.loteriaseresultados.com.br/index.php/megasena/concurso/'+ numero_concurso
    options = webdriver.FirefoxOptions()
    options.headless = True    
    driver = webdriver.Firefox(executable_path = r'/usr/local/bin/geckodriver', options = options)
    driver.get(url)
        
    try:
        body = driver.find_element_by_tag_name('body')
        html = body.get_attribute('innerHTML')
        soupPage = soup(html, 'lxml') # não conseguir utilizar a lib "html5lib" como solução usei a "lxml"
        spanList = soupPage.findAll("span", class_='white--text font-weight-bold')
        
        textCenter = soupPage.findAll('td', class_='text-center')
        textRight = soupPage.findAll('td', class_='text-right')
        
        numeros = []
        for n in range(len(spanList)):
            numeros.append(spanList[n].text)
        
  
        tb.insert_one({numero_concurso: {
                'Sena': numeros,
                textCenter[0].text: {'ganhadores': str(textCenter[1].text),'valor': str(textRight[0].text)},
                textCenter[2].text: {'ganhadores': str(textCenter[3].text),'valor': str(textRight[1].text)},
                textCenter[4].text: {'ganhadores': str(textCenter[5].text),'valor': str(textRight[2].text)}}
            })
    
    except Exception as Error:
        print(Error)
        print('nao encontrado')


    print('Terminou')
    driver.close()


In [8]:
getConcurso('2038')

list index out of range
nao encontrado
Terminou
